In [ ]:
!pip install matplotlib 

In [ ]:
from reprolab.experiment import start_experiment
start_experiment()

2025-09-19 15:29:30 - INFO - Starting experiment process
2025-09-19 15:29:30 - INFO - Step 1: Saving all notebooks
2025-09-19 15:29:30 - INFO - Attempting to save all Jupyter notebooks...
2025-09-19 15:29:30 - INFO - ipylab save command executed successfully
2025-09-19 15:29:30 - INFO - nbformat processing completed for 3 notebooks


In [ ]:
import random
random_integers = [random.randint(10, 90) for _ in range(10)]
correlated_random_integers = [x + random.randint(1, 10) for x in random_integers]

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(random_integers, correlated_random_integers)


In [ ]:
from reprolab.experiment import end_experiment
end_experiment()